# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai사이트에서 API KEY 발급
2. google colab - secret탭에 `OPENAI_API_KEY` 등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [5]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용


In [6]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 노동으로 끼니를 거르는 노동자들의 애환


In [7]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예상, 안전에 유의하세요"


## 연애코치 ReAct

In [9]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [10]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 어떻하죠?'))

1. 상황분석:
   - 100일 기념일은 연애의 중요한 이정표이며, 서로의 사랑을 기념하는 특별한 날입니다. 현재 당신은 여자친구와의 관계를 더욱 깊이 있게 만들고 싶어하는 것 같습니다. 이 기념일을 잘 기념하기 위한 고민이 드는 상황입니다. 여자친구의 취향, 기념일을 특별하게 만들어주는 요소 등이 중요한 포인트가 될 것입니다.

2. 행동계획:
   - 여자친구의 취향과 관심사를 고려하여 특별한 기념일 계획을 세우세요. 다음과 같은 요소를 고려하시면 좋습니다:
     - 선물: 개인적이고 의미 있는 선물 (예: 맞춤형 액세서리, 손편지, 사진 앨범 등)
     - 데이트: 특별한 장소에서의 저녁식사, 피크닉, 액티비티 (예: 함께 취미활동하기)
     - 경험: 둘만의 특별한 추억을 만들 수 있는 이벤트 (예: 영화 관람, 콘서트, 전시회)
   - 이런 요소들을 조합해서 여자친구가 감동받을 수 있는 기념일을 만들어 보세요. 

3. 실행:
   - 기념일 계획을 세운 후, 실행하기 전에 날짜와 시간을 조정해보세요. 
   - 필요한 준비물을 미리 준비하고, 예약이 필요한 경우는 사전 예약을 하세요. 
   - 가장 중요한 것은 여자친구에게 당신의 사랑과 진심을 표현하는 것입니다. 축하의 편지나 메모를 준비해 개인적인 메시지를 적어보세요. 
   - 기념일 당일에는 여유를 가지고 심플하게 즐거운 시간을 보내며 서로의 사랑을 깊이 느껴보세요. 

이제 기념일을 축하할 준비가 모두 완료되었습니다. 두 분의 100일이 더욱 특별하고 의미 있는 날로 기억되기를 바랍니다!


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면 LLM은 이를 바탕으로 어떤 음식을 만들지 조언해준다.

In [13]:
def recipe_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    냉장고에 있는 재료를 조합하여 자취생이 만들 수 있는 간단하지만 맛있는 요리 레시피를 추천해주세요.

    << Output Format >>
    1. 냉장고 속 재료 현황:

    2. 있는 재료를 확인하여 요리 추천:

    3. 레시피 보여주기:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [14]:
print(recipe_coach('지금 냉장고에 있는 재료는 계란, 닭가슴살, 양파 그리고 약간의 향신료들인데 어떤 음식을 만드는게 좋을까요?'))

1. 냉장고 속 재료 현황:
   - 계란
   - 닭가슴살
   - 양파
   - 향신료 (소금, 후추, 간장 등)

2. 있는 재료를 확인하여 요리 추천:
   - "닭가슴살 계란 볶음밥"을 추천합니다. 간단하면서도 맛있고 영양가 높은 요리입니다.

3. 레시피 보여주기:
   **닭가슴살 계란 볶음밥 레시피**

   **재료:**
   - 닭가슴살 100g (잘게 썬 것)
   - 계란 2개
   - 양파 1개 (작게 diced)
   - 향신료 (소금, 후추, 간장)

   **조리 방법:**
   1. 팬에 기름을 두르고 중불로 가열합니다.
   2. 썬 양파를 넣고 볶아 향이 날 때까지 볶습니다. 양파가 투명해지면 잘게 썬 닭가슴살을 추가합니다.
   3. 닭가슴살이 익고 노릇해질 때까지 볶습니다 (약 5-7분).
   4. 계란을 풀어 팬에 넣고, 원하는 만큼 잘 섞어줍니다.
   5. 마지막으로 소금, 후추, 간장으로 간을 맞춥니다. 노릇노릇하게 볶아내면 완성입니다.
   6. 뜨거운 밥 위에 얹어 즐기거나, 따로 접시에 담아도 좋습니다.

   이 요리는 간단하지만, 단백질과 영양이 풍부하여 자취생들에게 안성맞춤입니다. 맛있게 드세요!


## 면접질문 생성 JSON 출력


In [15]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [16]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
print(job_interview(job_posting))

{
    "hard_skill": [
        {
            "question": "Python 프로그래밍에서 가장 중요한 모듈이나 라이브러리는 무엇이며, 각각의 용도는 무엇인가요?",
            "answer": "Python에서 가장 중요한 라이브러리 중 하나는 NumPy입니다. 이는 고성능 수치 계산을 가능하게 해 주며, 배열 및 행렬 연산을 간편하게 처리할 수 있습니다. 또한, Pandas는 데이터 조작과 분석을 위한 훌륭한 라이브러리로, 데이터 프레임 구조를 통해 다양한 형태의 데이터셋을 쉽게 처리할 수 있습니다. 마지막으로, Scikit-learn은 머신러닝 모델을 구축하고 평가하는 데 유용하며, 다양한 알고리즘과 전처리 도구들을 제공해서 데이터 과학 작업을 수월하게 해줍니다."
        },
        {
            "question": "Kubernetes를 사용하여 머신러닝 모델을 배포하는 과정에 대해 설명해 주세요.",
            "answer": "Kubernetes를 사용한 머신러닝 모델 배포는 여러 단계로 진행됩니다. 먼저, Docker를 이용해 모델을 컨테이너화합니다. 그 다음, Kubernetes 클러스터에 배포하기 위해 YAML 파일을 작성하여 배포 구성(Deployment, Service 등)을 정의합니다. 이후 kubectl 명령어를 통해 클러스터에 배포하고, 이를 통해 모델에 대한 API를 생성합니다. 마지막으로, 필요한 경우 Helm을 통해 패키지 관리를 하거나, MLflow를 사용하여 모델의 버전 관리와 배포를 더욱 효율적으로 수행합니다."
        },
        {
            "question": "CI/CD를 이용한 머신러닝 프로젝트의 배포 자동화 프로세스를 설명해 주십시오.",
            "answer": "CI/CD를 활용한 머신러닝 프로젝트 배포 자동화는 다음과 같은 단계로 진행됩니다. 먼저, Git에 코드가 푸시되면 CI 

In [17]:
import json

# 응답 데이터가 json 형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)